In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import warnings

# Ignore warnings throughout the code
warnings.filterwarnings("ignore")

In [ ]:
# Chunking hop_teaming csv and adding it as a table to a SQLite DB
db = sqlite3.connect('data/healthcare_bluebook.sqlite')

for chunk in tqdm(pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv', chunksize = 100000)):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
    chunk = chunk[(chunk['transaction_count'] >= 50) & (chunk['average_day_wait'] < 50)]
    chunk.to_sql('hop', db, if_exists = 'append', index = False)            
                  
db.close()

In [9]:
# Chunking npi csv and adding it as a table to a SQLite DB
db = sqlite3.connect('data/healthcare_bluebook.sqlite')

for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20220213.csv', chunksize = 100000, dtype = str)):
    
    for counter in range(1,16):
        taxonomy_switch = chunk['Healthcare Provider Primary Taxonomy Switch_{}'.format(counter)] == 'Y'
        taxonomy_code_check = 'Healthcare Provider Taxonomy Code_{}'.format(counter)
        taxonomy_code = chunk.loc[taxonomy_switch, taxonomy_code_check]
        chunk.loc[taxonomy_switch, 'Healthcare Provider Taxonomy Code'] = taxonomy_code
    
    chunk = chunk[['NPI',
     'Entity Type Code',
     'Provider Organization Name (Legal Business Name)',
     'Provider Last Name (Legal Name)',
     'Provider First Name',
     'Provider Middle Name',
     'Provider Name Prefix Text',
     'Provider Name Suffix Text',
     'Provider Credential Text',
     'Provider First Line Business Practice Location Address',
     'Provider Second Line Business Practice Location Address',
     'Provider Business Practice Location Address City Name',
     'Provider Business Practice Location Address State Name',
     'Provider Business Practice Location Address Postal Code',
     'Healthcare Provider Taxonomy Code']]
    
    chunk.fillna(0)
    chunk['Provider Business Practice Location Address Postal Code'] = chunk['Provider Business Practice Location Address Postal Code'].str.zfill(9).str[:5]
    
    chunk.columns = [x.lower().replace(' ', '_').replace('(','').replace(')','') for x in chunk.columns]
    chunk.to_sql('npi', db, if_exists = 'append', index = False)            
                  
db.close()

In [6]:
# Chunking taxonomy csv and adding it as a table to a SQLite DB
db = sqlite3.connect('data/healthcare_bluebook.sqlite')

for chunk in tqdm(pd.read_csv('data/nucc_taxonomy_220.csv', chunksize = 100000)):
    
    chunk = chunk[['Code',
                   'Grouping',
                   'Classification']]
    
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
    chunk.to_sql('taxonomy_classification', db, if_exists = 'append', index = False)   
                  
db.close()

In [8]:
# Chunking cbsa csv and adding it as a table to a SQLite DB
db = sqlite3.connect('data/healthcare_bluebook.sqlite')

for chunk in tqdm(pd.read_csv('data/ZIP_CBSA_122021.csv', chunksize = 10000, dtype=str)):
    
    chunk = chunk[['zip', 
                   'cbsa', 
                   'usps_zip_pref_city', 
                   'usps_zip_pref_state']]
    
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
    chunk.to_sql('cbsa', db, if_exists = 'append', index = False)   
              
db.close()

In [13]:
db = sqlite3.connect('data/healthcare_bluebook.sqlite')

db.execute('CREATE INDEX from_npi ON hop(from_npi)')
db.execute('CREATE INDEX to_npi ON hop(to_npi)')
db.execute('CREATE INDEX npi_col ON npi(npi)')
db.execute('CREATE INDEX entity_type ON npi(entity_type_code)')
db.execute('CREATE INDEX city_name ON npi(provider_business_practice_location_address_city_name)')
db.execute('CREATE INDEX zip_code ON npi(provider_business_practice_location_address_postal_code)')
db.execute('CREATE INDEX taxonomy_code ON npi(healthcare_provider_taxonomy_code)')

db.close()

OperationalError: index city_name already exists

In [7]:
#Connecting to sqlite
conn = sqlite3.connect('data/healthcare_bluebook.sqlite')

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Doping EMPLOYEE table if already exists
cursor.execute("DROP TABLE cbsa")
print("Table dropped... ")

#Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()

Table dropped... 
